In [1]:
import requests 

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
import pandas as pd
import numpy as np

In [ ]:
browser = webdriver.Chrome()
browser.get("https://old.meneame.net/?page=128")

# pprint(browser.page_source)
soup = BeautifulSoup(browser.page_source, "html.parser")
browser.close()

In [ ]:
def extrae_datos_meneos (lista_meneos):
    datos = []
    continuar = True
    for meneo in lista_meneos:
        dicc_aux = {}

        try:
            dicc_aux["Comunidad"] = meneo.find("a", class_="subname").text.strip()
        except:
            dicc_aux["Comunidad"] = np.nan

        try:
            dicc_aux["Usuario"] = meneo.find("div", class_="news-submitted").find("img")["alt"]
        except:
            dicc_aux["Usuario"] = np.nan

        try:
            dicc_aux["Medio"] = meneo.find("span", class_="showmytitle").text.strip()
        except:            
            dicc_aux["Medio"] = np.nan
            
        try:
            fechas = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")
            dicc_aux["Enviado"] = fechas[0].text
            dicc_aux["Publicado"] = fechas[1].text
            if len(fechas[1].text) > 5:
                fech = datetime.strptime(fechas[1].text.split(" ")[0],"%d/%m/%Y")
                if fech.year == 2022:
                    continuar = False
        except:
            dicc_aux["Enviado"] = np.nan
            dicc_aux["Publicado"] = np.nan

        try:
            dicc_aux["Titular"] = meneo.find("h2").text.strip()
        except:
            dicc_aux["Titular"] = np.nan

        try:
            dicc_aux["Entradilla"] = meneo.find("div", class_="news-content").text.strip()
        except:
            dicc_aux["Entradilla"] = np.nan
    
        try:
            dicc_aux["Votos"] = meneo.find("div", class_="votes").find("a").text
        except:
            dicc_aux["Votos"] = np.nan

        try:
            dicc_aux["Clicks"] = meneo.find("div", class_="clics").find("span").text
        except:
            dicc_aux["Clicks"] = np.nan

        try:
            dicc_aux["Positivos"] = meneo.find("span", class_="positive-vote-number").text
        except:
            dicc_aux["Positivos"] = np.nan

        try:
            dicc_aux["Anonimos"] = meneo.find("span", class_="anonymous-vote-number").text
        except:
            dicc_aux["Anonimos"] = np.nan

        try:
            dicc_aux["Negativos"] = meneo.find("span", class_="negative-vote-number").text
        except:
            dicc_aux["Negativos"] = np.nan

        try:
            dicc_aux["Comentarios"] = meneo.find("a", class_="comments")["data-comments-number"]
        except:
            dicc_aux["Comentarios"] = np.nan

        try:
            dicc_aux["Karma"] = meneo.find("span", class_="karma-number").text
        except:
            dicc_aux["Karma"] = np.nan
        
        datos.append(dicc_aux)        

    return [datos, continuar]

In [ ]:
########################### BLOQUE PARA PRUEBAS ###################################
########################### Cambiar número de página y condición del contador ###################################

browser = webdriver.Chrome()
pagina = 725
lista_aux = []
contador = 0

while contador < 10 and continuar == True:
    try:
        browser.get(f"https://old.meneame.net/?page={pagina}")        
        Soup_pagina_meneos = BeautifulSoup(browser.page_source, "html.parser")
        bloques_meneos = Soup_pagina_meneos.find_all("div", class_="news-summary")
        resultado = extrae_datos_meneos(bloques_meneos)    
        lista_aux.extend(resultado[0])
        continuar = resultado[1]
        pagina +=1
        contador += 1
    except:
        print(f"Error en página {pagina}")
    # sleep(random.randint(1,3))

print(f"Total paginas importadas: {contador}")

browser.close()